<a href="https://colab.research.google.com/github/GermanM3/GermanM3/blob/master/%EC%86%8C%ED%98%95%EC%A3%BC_%EB%A7%88%EB%B2%95%EA%B3%B5%EC%8B%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import datetime
import time
import random
from concurrent.futures import ThreadPoolExecutor

In [2]:
from google.colab import files

# 파일 업로드
uploaded = files.upload()
# 파일명 확인 (업로드한 파일명을 딕셔너리로 반환)
file_name = list(uploaded.keys())[0]

# 엑셀 파일 읽기
df = pd.read_excel(file_name)
df

Saving Fundamental.xlsx to Fundamental.xlsx


,Date,code,BPS,PER,PBR,EPS,DIV,DPS,ROE
0,2014-01-31,14280,40612,6.97,0.41,2396,2.10,350,0.058997
1,2014-02-28,14280,40612,8.06,0.48,2396,1.81,350,0.058997
2,2014-03-31,14280,40612,10.91,0.64,2396,1.34,350,0.058997
3,2014-04-30,14280,40612,13.86,0.82,2396,1.05,350,0.058997
4,2014-05-31,14280,42876,11.71,0.98,3586,0.83,350,0.083637
...,...,...,...,...,...,...,...,...,...
79539,2024-05-31,96870,2287,0.00,1.38,0,0.00,0,0.000000
79540,2024-06-30,96870,2287,0.00,1.25,0,0.00,0,0.000000
79541,2024-07-31,96870,2287,0.00,1.23,0,0.00,0,0.000000
79542,2024-08-31,96870,2287,0.00,1.22,0,0.00,0,0.000000


In [4]:
df['code']=df['code'].astype(str).str.zfill(6)

In [3]:
from google.colab import files

# 파일 업로드
uploaded = files.upload()
# 파일명 확인 (업로드한 파일명을 딕셔너리로 반환)
file_name = list(uploaded.keys())[0]

# 엑셀 파일 읽기
mret = pd.read_excel(file_name)
mret

Saving 1000억이하 월말종가.xlsx to 1000억이하 월말종가.xlsx


,Date,014280,370090,111380,145170,418470,037070,102370,353810,140070,...,335870,196700,051780,053060,121890,069330,106240,084440,011080,096870
0,2014-01-31,4020,NaN,NaN,NaN,NaN,4002,NaN,NaN,NaN,...,NaN,NaN,1010,2359,2116,20100,788,5100,3296,2785
1,2014-02-28,5300,NaN,NaN,NaN,NaN,4644,NaN,NaN,NaN,...,NaN,NaN,1160,3746,2014,21500,876,2170,3183,2650
2,2014-03-31,6250,NaN,NaN,NaN,NaN,4683,NaN,NaN,NaN,...,NaN,NaN,1065,3299,2025,20800,854,1800,3979,2335
3,2014-04-30,8370,NaN,NaN,NaN,NaN,5325,NaN,NaN,NaN,...,NaN,NaN,992,3184,2179,21400,1009,2385,3693,2860
4,2014-05-31,10460,NaN,NaN,NaN,NaN,6945,NaN,NaN,NaN,...,NaN,NaN,1275,3104,2240,17400,1013,2190,3693,2580
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,2024-05-31,5050,22300.0,26600.0,37400.0,19480.0,8770,3965.0,4250.0,3410.0,...,1364.0,1532.0,346,1417,1875,1511,1313,1115,782,2865
125,2024-06-30,4970,20550.0,22750.0,24650.0,17090.0,10560,3485.0,4455.0,3365.0,...,1310.0,1515.0,351,1427,1884,1402,1237,988,630,2855
126,2024-07-31,5140,16790.0,19960.0,16500.0,16810.0,9170,3705.0,4470.0,3095.0,...,1284.0,1600.0,312,1280,1921,1504,1176,964,650,2790
127,2024-08-31,4685,15730.0,17840.0,17160.0,16350.0,6300,3210.0,3850.0,3010.0,...,1215.0,1371.0,248,1144,1690,1294,1155,833,616,2495


In [25]:
price_data= mret.set_index('Date')

In [45]:
# 자본수익률(ROC)과 이익수익률 계산
df['ROC'] = df['ROE']  # ROE를 그대로 사용
df['Earnings_Yield'] = 1 / df['PER']  # PER의 역수

# 종목별로 그룹화하여 각 월별 ROC와 이익수익률의 순위 계산
df['ROC_Rank'] = df.groupby('Date')['ROC'].rank(ascending=False)
df['EY_Rank'] = df.groupby('Date')['Earnings_Yield'].rank(ascending=False)

# 마법공식 순위: ROC와 이익수익률 순위의 합
df['Magic_Formula_Rank'] = df['ROC_Rank'] + df['EY_Rank']

# 각 월별로 상위 n개 종목 선택 (예: n = 5)
n = 5
top_n_stocks = df.groupby('Date').apply(lambda x: x.nsmallest(n, 'Magic_Formula_Rank')).reset_index(drop=True)

print(top_n_stocks[['Date', 'code', 'Magic_Formula_Rank']])

          Date    code  Magic_Formula_Rank
0   2014-01-31  005320               108.0
1   2014-01-31  037350               119.0
2   2014-01-31  079650               119.0
3   2014-01-31  039290               120.0
4   2014-01-31  153460               140.0
..         ...     ...                 ...
640 2024-09-30  154040               180.0
641 2024-09-30  042940               181.0
642 2024-09-30  092460               184.0
643 2024-09-30  085310               184.0
644 2024-09-30  019540               195.0

[645 rows x 3 columns]


<ipython-input-45-7e7ef33a4e85>:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_n_stocks = df.groupby('Date').apply(lambda x: x.nsmallest(n, 'Magic_Formula_Rank')).reset_index(drop=True)


In [46]:
# 각 월별로 상위 n개 종목 선택 (예: n = 5)
n = 5
top_n_stocks = df.groupby('Date').apply(lambda x: x.nsmallest(n, 'Magic_Formula_Rank')).reset_index(drop=True)

def backtest(price_data, top_n_stocks, n=5):
    # 월별 수익률 저장 변수 초기화
    monthly_returns = []

    # 월별로 반복하여 상위 n개의 종목에 대한 수익률 계산
    for date in top_n_stocks['Date'].unique():
        # 해당 월의 상위 n 종목 가져오기
        top_stocks = top_n_stocks[top_n_stocks['Date'] == date]['code'].values

        # 상위 종목의 가격 데이터 필터링
        if len(top_stocks) == 0:
            continue

        # 현재 월 가격
        try:
            prices = price_data.loc[date, top_stocks]
        except KeyError:
            print(f"가격 데이터가 존재하지 않음: {date}에 대한 {top_stocks}")
            continue

        # 수익률 계산을 위한 직전 월 가격 데이터
        prev_date = price_data.index[price_data.index < date].max()
        if pd.isna(prev_date):  # NaT가 아닌 경우만 진행
            print(f"유효한 이전 날짜가 없습니다: {date}")
            continue

        try:
            prev_prices = price_data.loc[prev_date, top_stocks]
            # 수익률 계산 (현재 월 / 이전 월 - 1)
            returns = (prices / prev_prices - 1).mean()
            monthly_returns.append((date, returns))
        except KeyError:
            print(f"이전 가격 데이터가 존재하지 않음: {prev_date}에 대한 {top_stocks}")
            continue

    # 결과 데이터프레임 생성
    monthly_returns_df = pd.DataFrame(monthly_returns, columns=['Date', 'Monthly_Return'])
    monthly_returns_df.set_index('Date', inplace=True)

    # 누적 수익률 계산
    cumulative_return = (1 + monthly_returns_df['Monthly_Return']).cumprod() - 1

    return monthly_returns_df, cumulative_return

# 백테스팅 실행
monthly_returns_df, cumulative_return = backtest(price_data, top_n_stocks, n=3)

유효한 이전 날짜가 없습니다: 2014-01-31 00:00:00
